In [2]:
import mri_helpers as mh
import SimpleITK as sitk
import glob
from ipywidgets import IntProgress
import IPython

In [3]:
BRATS_MRI_REF = '../../datasets/BRATS2015/BRATS2015_Training/LGG/brats_2013_pat0001_1/VSD.Brain.XX.O.MR_T1.54633/VSD.Brain.XX.O.MR_T1.54633.mha'
BRATS_ROI_REF = '../../datasets/BRATS2015/BRATS2015_Training/LGG/brats_2013_pat0001_1/VSD.Brain_3more.XX.O.OT.54637/VSD.Brain_3more.XX.O.OT.54637.mha'
BD2DECIDE_INPUT_FOLDER = '../../datasets/BD2Decide/T1T2/'
BD2DECIDE_OUTPUT_FOLDER = 'resampled/BD2Decide/T1T2_reshaped/'


In [4]:
input_file_list = glob.glob(BD2DECIDE_INPUT_FOLDER+'*/*.mha')
input_files = [(in_path, BD2DECIDE_OUTPUT_FOLDER+in_path.split(BD2DECIDE_INPUT_FOLDER)[-1] ,'ROI' in in_path) for in_path in input_file_list]

In [5]:
progressbar = IntProgress(value=0, min=0, max=len(input_files)-1, step=1)
display(progressbar)
for i, (in_path, out_path, is_label) in enumerate(input_files):
    REF = BRATS_ROI_REF if is_label else BRATS_MRI_REF
    mh.resample_to(in_path, REF, out_path, is_label)
    progressbar.value=i
    
    

IntProgress(value=0, max=1427)

In [6]:
processed_file_list = glob.glob(BD2DECIDE_OUTPUT_FOLDER+'*/*.mha')
print("Input files to process: {}".format(len(input_file_list)))
print("Processed files: {}".format(len(processed_file_list)))

Input files to process: 1428
Processed files: 1428


### Check which samples have T1 or T2

In [61]:
folders = glob.glob('../../datasets/BD2Decide/resampled/T1T2_reshaped/*')
import pandas as pd
import os
pdf = pd.DataFrame(folders, columns=['folder'])

In [62]:
check_files= lambda x, s: any([s in os.path.basename(f) for f in glob.glob(x+'/*.mha')])
check_t1 = lambda x: check_files(x, s='T1')
check_t2 = lambda x: check_files(x, s='T2')

In [63]:
pdf['has_T1'] = pdf['folder'].map(check_t1)
pdf['has_T2'] = pdf['folder'].map(check_t2)

In [72]:
len(folders)

486

In [87]:
no_t1 = pdf[~pdf['has_T1']]['folder'].values
no_t2 = pdf[~pdf['has_T2']]['folder'].values


In [97]:
# Print a bash-for to move all the folders out of the directory
foldlist = ""
for f in no_t1:
    foldlist += f.split('/')[-1] + " "
for f in no_t2:
    foldlist += f.split('/')[-1] + " "
foldlist
command = "for f in {}; do sudo mv $f ../T1T2_missingdata/ ; done".format(foldlist)
command


'for f in SS_INT_0012 SS_INT_0074 SS_INT_0077 SS_INT_0119 SS_INT_0160 SS_INT_0188 SS_INT_0213 SS_INT_0308 SS_INT_0312 SS_INT_0315 SS_INT_0324 SS_INT_0337 SS_INT_0370 SS_INT_0382 SS_INT_0435 SS_INT_0486 SS_INT_MI_0195 SS_PR130 SS_134 SS_176 SS_177 SS_227 SS_93_9405 SS_INT_0068 SS_NPR125 SS_PR132 SS_PR139 SS_PR147 SS_PR152 SS_PR154 ; do sudo mv $f ../T1T2_missingdata/ ; done'

In [89]:
for missing in no_t1:
    mri_folder = missing.split('/')[-1]
    shutil.move(missing, INCOMPLETE_FOLDER)

OSError: [Errno 30] Read-only file system: 'MR_T2_TSE.mha'